In [1]:
!pip install 


ERROR: You must give at least one requirement to install (see "pip help install")


In [2]:
!pip install jupyterthemes


In [2]:
!jt -t onedork


Der Befehl "jt" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [1]:
import pandas as pd
import xlrd
import re

In [2]:
file_path= "2021-06-19-cartagena-casa-arriendo-MC-details.xlsm"
sheet_index = "2021-06-19-cartagena-casa-arrie"
workbook = xlrd.open_workbook(file_path)
worksheet = workbook.sheet_by_name(sheet_index)

In [3]:
number_col = worksheet.ncols;
number_rows = worksheet.nrows;

In [4]:
number_col = worksheet.ncols;
number_rows = worksheet.nrows;
print("number of cols ",number_col)
print("number of rows ", number_rows)

number of cols  1
number of rows  194


In [5]:
firstRow = worksheet.row(0)
firstCol = worksheet.col(0)

In [6]:
x=[]
for cell in range(number_rows):
    row = str(worksheet.row(cell)) 
    row1 = re.sub("text:'","", row)
    row1 = re.sub('\[', "", row1)
    row1 = re.sub('\]', "", row1)
    row1 = re.sub('\'', "", row1)
    #row1 = re.sub('\"', "", row1)
    row1 = re.sub('Casa,"y Arriendo', "Casa y Arriendo", row1)
    row1 = re.sub('Ã³', "o", row1)
    row1 = re.sub('Ã¼', "u", row1)
    row1 = re.sub('Ãº', "u", row1)
    row1 = re.sub('x81', "A", row1)
    row1 = re.sub("Ã¡", "a", row1)
    row1 = re.sub("Ã±", "n", row1)
    row1 = re.sub(r'\\"', "\]", row1)
    row1 = re.sub(r'"\\', r"[", row1)
    row1 = re.sub(r'\\', r"", row1)
    row1 = re.sub(r',"', r",[", row1)
    row1 = re.sub(r'",,,,', r"]", row1)
    row1 = re.sub(r'"', r"", row1)
    row1 = re.sub(r',lat,long,id,date,category,location,mode,price,details,description,surface,rooms,baths,park', r"house_ID,lat,long,id,date,category,location,mode,price,details,description", row1)
    row1 = re.sub('ÃArea', "Area", row1)
    row1 = re.sub(', ', ",", row1)
    row1 = re.sub(', ', ",", row1)
    #row1 = re.sub('\"..",', "[..]", row1)
    res = re.split(r',(?![^\[\]]*\])', row1)
    x.append(res)


In [7]:
res1=[]
for a, b in enumerate(x):
    res= x[a][9]
    res = re.sub("\[|\]", "", res)
    res= re.split('({}\s[^,]+)'.format("Codigo"),res)
    res1.append(res)
    
print(res1)

[['details'], ['', 'Codigo inmueble847-5550L2', ',Barrio comunBAHIA MANZANILLO,Precio$2.150.000.000,AntiguedadEntre 10 y 20 anos,Area construida283 mÂ²,Area privada0 mÂ²,Valor arriendo$20.000.000,Valor administracion$810.000,Parqueaderos2,Area construida 283 mÂ²,3Habitaciones,3Banos,6Estrato'], ['', 'Codigo inmueble3325-M2909989', ',Barrio comunMANGA,AntiguedadEntre 10 y 20 anos,Area construida84 mÂ²,Area privada84 mÂ²,Valor arriendo$2.000.000,Area construida 84 mÂ²,3Habitaciones,1Banos,5Estrato'], ['', 'Codigo inmueble37-3422', ',Barrio comunTorices,Precio$1.100.000.000,AntiguedadEntre 10 y 20 anos,Area construida800 mÂ²,Area privada596 mÂ²,Valor arriendo$6.000.000,Parqueaderos4,Area construida 800 mÂ²,1Habitaciones,4Estrato'], ['', 'Codigo inmueble11673-M2695961', ',Barrio comunARROYO DE PIEDRA,Precio$1.100.000.000,AntiguedadEntre 5 y 10 anos,Area construida200 mÂ²,Area privada200 mÂ²,Valor arriendo$8.000.000,Parqueaderos3,Area construida 200 mÂ²,3Habitaciones,3Banos,6Estrato'], ['',

In [8]:

Codigo_Inmueble1=[]
Lineas_a_eliminar=[]
Codigo_Inmuebles_Diccionario= {}
i=0
for a, b in enumerate(res1):
    try:
        Codigo_Inmueble =(res1[a][1])
        res = re.sub("([a-z])([\d])", r"\1;\2", Codigo_Inmueble)
        res = re.split(";", res)
        objeto =res[0] 
        valor = str(res[1])
        
        #print(a, res1[a][1])
        #print(res)
    except IndexError:
        Lineas_a_eliminar.append(a)
        objeto = "Codigo inmueble"
        valor = "NaN"
    Codigo_Inmuebles_Diccionario.setdefault(objeto, [])
    Codigo_Inmuebles_Diccionario[objeto].append(valor)
    


In [9]:
df_C = pd.DataFrame(Codigo_Inmuebles_Diccionario[objeto], columns = [objeto])
df_C= df_C.iloc[1:]
print(df_C.columns[0])

Codigo inmueble


In [10]:
res_list3 =[]
max_len_dummy = 0
for a, b in enumerate(res1):
    try:
        res2 = res1[a][2]
        res2 = re.sub("{}".format("construida "), "construida", res2)
        res2 = re.sub("\$", "", res2)
        res2 = re.sub("(\d)([A-Z][a-z]+)", r"\2\1", res2)
        res2 = re.sub("([a-z])([A-Z\d])", r"\1;\2", res2)
        
        res2 = re.split(",", res2)
        res2.pop(0)
        if len(res2)>max_len_dummy:
            dummy =[]
            max_len_dummy= len(res2)
            dummy = res2
        #print(res2)
    except IndexError:
        res2 = "NaN"
    res_list3.append(res2)
print(dummy)


['Barrio comun;BAHIA MANZANILLO', 'Precio;2.150.000.000', 'Antiguedad;Entre 10 y 20 anos', 'Area construida;283 mÂ²', 'Area privada;0 mÂ²', 'Valor arriendo;20.000.000', 'Valor administracion;810.000', 'Parqueaderos;2', 'Area construida;283 mÂ²', 'Habitaciones;3', 'Banos;3', 'Estrato;6']


In [14]:
dictionary_dummy={}

dictionary_res3 = {}
List_dummy3 =[]
for i in dummy:
    i = re.findall("[A-Z][^;]+", i)
    dictionary_dummy.setdefault(i[0],None)

for a, b in enumerate(res_list3):
    #print(b)
    dictionary_dummy2={}
    dictionary_dummy2.update(dictionary_dummy)
    
    if b != "NaN":
        for i in b:
            try:
                res3 = re.split(";", i)
                #print(res3)
                parameter = res3[0]
                valor     = res3[1]
                #dictionary_dummy2[parameter] = (valor)
            except IndexError:
                parameter = res3[0]
                valor     = "NaN"
                #dictionary_res3[parameter] = (valor)  
            dictionary_dummy2[parameter] = (valor)
        #dictionary_dummy2.update(dictionary_res3)
        print("the first one is ", dictionary_dummy2["Barrio comun"])
        print(dictionary_dummy2)
        List_dummy3.append(dictionary_dummy2)

the first one is  BAHIA MANZANILLO
{'Barrio comun': 'BAHIA MANZANILLO', 'Precio': '2.150.000.000', 'Antiguedad': 'Entre 10 y 20 anos', 'Area construida': '283 mÂ²', 'Area privada': '0 mÂ²', 'Valor arriendo': '20.000.000', 'Valor administracion': '810.000', 'Parqueaderos': '2', 'Habitaciones': '3', 'Banos': '3', 'Estrato': '6'}
the first one is  MANGA
{'Barrio comun': 'MANGA', 'Precio': None, 'Antiguedad': 'Entre 10 y 20 anos', 'Area construida': '84 mÂ²', 'Area privada': '84 mÂ²', 'Valor arriendo': '2.000.000', 'Valor administracion': None, 'Parqueaderos': None, 'Habitaciones': '3', 'Banos': '1', 'Estrato': '5'}
the first one is  Torices
{'Barrio comun': 'Torices', 'Precio': '1.100.000.000', 'Antiguedad': 'Entre 10 y 20 anos', 'Area construida': '800 mÂ²', 'Area privada': '596 mÂ²', 'Valor arriendo': '6.000.000', 'Valor administracion': None, 'Parqueaderos': '4', 'Habitaciones': '1', 'Banos': None, 'Estrato': '4'}
the first one is  ARROYO DE PIEDRA
{'Barrio comun': 'ARROYO DE PIEDRA', 

In [15]:
List_dummy3

[{'Barrio comun': 'BAHIA MANZANILLO',
  'Precio': '2.150.000.000',
  'Antiguedad': 'Entre 10 y 20 anos',
  'Area construida': '283 mÂ²',
  'Area privada': '0 mÂ²',
  'Valor arriendo': '20.000.000',
  'Valor administracion': '810.000',
  'Parqueaderos': '2',
  'Habitaciones': '3',
  'Banos': '3',
  'Estrato': '6'},
 {'Barrio comun': 'MANGA',
  'Precio': None,
  'Antiguedad': 'Entre 10 y 20 anos',
  'Area construida': '84 mÂ²',
  'Area privada': '84 mÂ²',
  'Valor arriendo': '2.000.000',
  'Valor administracion': None,
  'Parqueaderos': None,
  'Habitaciones': '3',
  'Banos': '1',
  'Estrato': '5'},
 {'Barrio comun': 'Torices',
  'Precio': '1.100.000.000',
  'Antiguedad': 'Entre 10 y 20 anos',
  'Area construida': '800 mÂ²',
  'Area privada': '596 mÂ²',
  'Valor arriendo': '6.000.000',
  'Valor administracion': None,
  'Parqueaderos': '4',
  'Habitaciones': '1',
  'Banos': None,
  'Estrato': '4'},
 {'Barrio comun': 'ARROYO DE PIEDRA',
  'Precio': '1.100.000.000',
  'Antiguedad': 'Entre 5

In [27]:
df_dicts = pd.DataFrame.from_records(List_dummy3)
df_dicts = df_dicts.iloc[:,:11]
df_dicts

,Barrio comun,Precio,Antiguedad,Area construida,Area privada,Valor arriendo,Valor administracion,Parqueaderos,Habitaciones,Banos,Estrato
0,BAHIA MANZANILLO,2.150.000.000,Entre 10 y 20 anos,283 mÂ²,0 mÂ²,20.000.000,810.000,2,3,3,6
1,MANGA,None,Entre 10 y 20 anos,84 mÂ²,84 mÂ²,2.000.000,None,None,3,1,5
2,Torices,1.100.000.000,Entre 10 y 20 anos,800 mÂ²,596 mÂ²,6.000.000,None,4,1,None,4
3,ARROYO DE PIEDRA,1.100.000.000,Entre 5 y 10 anos,200 mÂ²,200 mÂ²,8.000.000,None,3,3,3,6
4,CRESPO,1.300.000.000,Entre 10 y 20 anos,0 mÂ²,390 mÂ²,4.000.000,None,None,3,4,5
...,...,...,...,...,...,...,...,...,...,...,...
187,BOCAGRANDE,None,Entre 0 y 5 anos,500 mÂ²,0 mÂ²,25.000.000,None,None,5,5,6
188,MANGA,None,Entre 10 y 20 anos,144 mÂ²,0 mÂ²,4.000.000,None,2,3,3,None
189,Toril,None,Entre 10 y 20 anos,208 mÂ²,208 mÂ²,3.000.000,None,None,5,5,2
190,Castillogrande,None,Entre 10 y 20 anos,339.59 mÂ²,339.59 mÂ²,7.500.000,720.000,1,3,4,6


In [16]:
df = pd.DataFrame(x)
df.columns = df.iloc[0] 
df = df[1:]
df = df.iloc[:, 0:11]
df[df_C.columns[0]]= df_C
df.head()

,house_ID,lat,long,id,date,category,location,mode,price,details,description,Codigo inmueble
1,0,10.420338,-75.54401,847-5550L2,2021-06-19,Casa y Arriendo,BAHIA MANZANILLO,Venta y Arriendo,$2.150.000.000,"[Codigo inmueble847-5550L2,Barrio comunBAHIA M...",[EnPADStepresentamosenventaunaespectacularcasa...,847-5550L2
2,1,10.41117,-75.53565,3325-M2909989,2021-06-19,Casa,MANGA,Arriendo,$2.000.000,"[Codigo inmueble3325-M2909989,Barrio comunMANG...","[OBSERVACIONESINTERNAS:CASA1NIVEL,EXCELENTESAC...",3325-M2909989
3,2,10.431309,-75.533066,37-3422,2021-06-19,Casa y Arriendo,Torices,Venta y Arriendo,$1.100.000.000,"[Codigo inmueble37-3422,Barrio comunTorices,Pr...","[Ampliacasadediezalcobas,nuevebanos,patioconar...",37-3422
4,3,0,0,11673-M2695961,2021-06-19,Casa y Arriendo,ARROYO DE PIEDRA,Venta y Arriendo,$1.100.000.000,"[Codigo inmueble11673-M2695961,Barrio comunARR...","[Vendoyalquilo,estshermosacasa,consalidadirect...",11673-M2695961
5,4,10.443557,-75.519226,671-72690,2021-06-19,Casa y Arriendo,CRESPO,Venta y Arriendo,$1.300.000.000,"[Codigo inmueble671-72690,Barrio comunCRESPO,P...","[CASAAMPLIACONTRESALCOBAS.CUATROBAÃ‘OS,SALACOM...",671-72690


In [17]:
print(df)

0   house_ID         lat        long              id        date  \
1          0   10.420338   -75.54401      847-5550L2  2021-06-19   
2          1    10.41117   -75.53565   3325-M2909989  2021-06-19   
3          2   10.431309  -75.533066         37-3422  2021-06-19   
4          3           0           0  11673-M2695961  2021-06-19   
5          4   10.443557  -75.519226       671-72690  2021-06-19   
..       ...         ...         ...             ...         ...   
189      188    10.40206  -75.555664       671-81837  2021-06-19   
190      189   10.393228   -75.48323      2398-22795  2021-06-19   
191      190   10.419311  -75.529366    411-M2886538  2021-06-19   
192      191    10.39447   -75.55161    411-M2886194  2021-06-19   
193      192  10.4242735  -75.549515    3967-1584515  2021-06-19   

0           category          location              mode           price  \
1    Casa y Arriendo  BAHIA MANZANILLO  Venta y Arriendo  $2.150.000.000   
2               Casa           